Importing the libraries

In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf


  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached absl_py-0.10.0-py3-none-any.whl (127 kB)
  Using cached google_auth-1.22.1-py2.py3-none-any.whl (114 kB)
  Using cached google_auth_oauthlib-0.4.1-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_plugin_wit-1.7.0-py3-none-any.whl (779 kB)
  Using cached Markdown-3.3.2-py3-none-any.whl (95 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached rsa-4.6-py3-none-any.whl (47 kB)
  Using cached cachetools-4.1.1-py3-none-any.whl (10 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4835 sha256=739560851e2f459aaa86a8efeb7702bb5913e3b67f6093aec78b0f04ae75fbe4
  Stored in directory: c:\users\muranaher\appdata\local\pip\cach

In [17]:
tf.__version__

'2.3.1'

Part 1: Data Preprocessing

In [18]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [19]:
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


In [20]:
#Do label encoding for the gender column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [21]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [22]:
#Do one hot encoding for the geography column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [23]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [24]:
#Split the dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state=0)

In [25]:
#Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Part 2: Building the ANN

In [26]:
#Initializing the ANN as a sequence of layers
ann = tf.keras.models.Sequential()
#Adding the input layer (automatically) and the first hidden layer
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))
#Second hidden layer
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))
#Output layer
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

Part 3: Training the ANN

In [27]:
#Compile the ANN
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [28]:
#Training the ANN on the Training set
ann.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 916us/step - loss: 0.5268 - accuracy: 0.7837
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4638 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 916us/step - loss: 0.4457 - accuracy: 0.7968
Epoch 4/100
250/250 [==============================] - 0s 700us/step - loss: 0.4368 - accuracy: 0.7980
Epoch 5/100
250/250 [==============================] - 0s 796us/step - loss: 0.4304 - accuracy: 0.7999
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4249 - accuracy: 0.8037
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4191 - accuracy: 0.8052
Epoch 8/100
250/250 [==============================] - 0s 808us/step - loss: 0.4129 - accuracy: 0.8069
Epoch 9/100
250/250 [==============================] - 0s 720us/step - loss: 0.4065 - accuracy: 0.8091
Epoch 10/100
250/250 [==============================] - 0s 748us/step - loss: 0

Part 4: Making the predictions and evaluating the model

In [29]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

[[False]]


In [30]:
#Predicting the Test set results
y_pred = ann.predict(X_test)
y_pred = (y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [31]:
#Creating the confusion matrix
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[1516   79]
 [ 198  207]]


0.8615